## [ 웹크롤링 _ 나무위키 사이트 분석 및 시각화 ]

### <Step1. 크롤링> : 크롤링으로 웹 데이터 가져오기

[웹크롤링 라이브러리 사용하기]
- 파이썬에서는 BeautifulSoup과 requests라는 라이브러리로 웹 크롤러를 만들 수 있음
- requests는 특정 URL로부터 HTML 문서를 가져오는 작업을 수행
- 나무위키와 같은 페이지는 HTML 문서가 Javascript로 동적 로딩되는 경우가 있음
- requests 대신 셀레니움(selenium) 라이브러리를 이용해 크롬 브라우저로 동적 웹크롤링 수행
- selenium은 웹 브라우저를 자동으로 구동해주는 라이브러리
- selenium을 사용하기 위해 크롬 드라이버를 이용해 크롬 브라우저 자동으로 구동=> 크롬드라이버 필요

### [BeautifulSoup과 selenium을 이용한 웹 크롤링]
- anaconda prompt 혹은 Terminal에서 아래와 같은 패키지들을 설치
- (env_name) pip install selenium
- (env_name) pip install beautifulsoup4

### [크롬 브라우저 업데이트 및 크롬 드라이버 설치]
- 크롬 브라우저 설정에서 최신 버전으로 업데이트
- 크롬 드라이버 사이트에서 브라우저 버전에 맞는 드라이버 다운로드
  - https://chromedriver.chromium.org/downloads
- chromedriver.exe 파일을 노트북 파일 경로에 이동

In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### [ BeautifulSoup의 select() VS find_all() ]
- HTML의 특정 요소 선택
- select, select_one 의 경우 CSS 선택자를 이용하는 것처럼 사용 가능
- select의 경우 후손이나 자손 요소를 CSS 처럼 선택 가능
- 예) soup.select("dl > dt > a") 
- find_all, find 의 경우 하나의 태그(name="table")나 하나의 클래스(class="tables")를 선택
- find의 경우 후손이나 자손 요소를 직접 선택할 수 없어 한번 더 변수에 담든지 루프 문을 이용해야 함
- 예) find_all(class="ah_roll"), find(name="table")

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re # 정규식 표현을 위한 모듈


# 윈도우용 크롬 웹드라이버 실행 경로 (Windows) 지정
excutable_path = "chromedriver.exe"
driver = webdriver.Chrome(executable_path=excutable_path)

# 사이트의 html 구조에 기반하여 크롤링을 수행
source_url = "https://namu.wiki/RecentChanges" # 크롤링할 사이트 주소를 정의
driver.get(source_url)  # 크롬 드라이버를 통해 URL의 HTML 문서 가져옴

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "app")))

req = driver.page_source
soup = BeautifulSoup(req, "html.parser") # BeautifulSoup의 soup 객체로 변환

#contents_table = soup.find(name="table")  
#table_body = contents_table.find(name="tbody")
#table_rows = table_body.find_all(name="tr")
table_rows = soup.select("table tbody tr")

### [페이지 링크주소 리스트 가져오기]

In [3]:
page_url_base = "https://namu.wiki" # 베이스 URL 정의
page_urls = [] # href 속성값을 담기 위한 빈 리스트 생성 

for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all("td")[0]
    td_url = first_td.find_all("a")
    if len(td_url) > 0:
        # 특정 속성 선택시 attrs["속성명"] 또는 get("속성명") 사용
        # page_url = page_url_base + td_url[0].get("href")
        # attrs는 딕셔너리 형태로 속성명과 속성값을 불러옴
        # attrs["href"]는 attrs 결과 중 key가 href인 것의 값만 불러옴
        page_url = page_url_base + td_url[0].attrs["href"] 
        if "png" not in page_url:
            page_urls.append(page_url)
            print(page_urls)

['https://namu.wiki/w/%ED%9C%98%EC%9E%A5']
['https://namu.wiki/w/%ED%9C%98%EC%9E%A5', 'https://namu.wiki/w/%EC%A7%80%EC%96%B8%ED%95%99']
['https://namu.wiki/w/%ED%9C%98%EC%9E%A5', 'https://namu.wiki/w/%EC%A7%80%EC%96%B8%ED%95%99', 'https://namu.wiki/w/%EB%B8%8C%EB%A0%8C%ED%8A%B8%ED%8F%AC%EB%93%9C%20FC/2021-22%20%EC%8B%9C%EC%A6%8C/%EB%A6%AC%EA%B7%B8']
['https://namu.wiki/w/%ED%9C%98%EC%9E%A5', 'https://namu.wiki/w/%EC%A7%80%EC%96%B8%ED%95%99', 'https://namu.wiki/w/%EB%B8%8C%EB%A0%8C%ED%8A%B8%ED%8F%AC%EB%93%9C%20FC/2021-22%20%EC%8B%9C%EC%A6%8C/%EB%A6%AC%EA%B7%B8', 'https://namu.wiki/w/%EA%B4%B4%EB%AC%BC%EC%82%AC%EB%B3%80']
['https://namu.wiki/w/%ED%9C%98%EC%9E%A5', 'https://namu.wiki/w/%EC%A7%80%EC%96%B8%ED%95%99', 'https://namu.wiki/w/%EB%B8%8C%EB%A0%8C%ED%8A%B8%ED%8F%AC%EB%93%9C%20FC/2021-22%20%EC%8B%9C%EC%A6%8C/%EB%A6%AC%EA%B7%B8', 'https://namu.wiki/w/%EA%B4%B4%EB%AC%BC%EC%82%AC%EB%B3%80', 'https://namu.wiki/w/%ED%86%A0%ED%8A%B8%EB%84%98%20%ED%99%8B%EC%8A%A4%ED%8D%BC%20FC/2021-22%20%

In [4]:
td_url[0].attrs

{'data-v-4406da71': '',
 'href': '/w/%ED%95%B4%EB%A6%AC%20%EC%BC%80%EC%9D%B8/%ED%81%B4%EB%9F%BD%20%EA%B2%BD%EB%A0%A5'}

In [5]:
page_urls

['https://namu.wiki/w/%ED%9C%98%EC%9E%A5',
 'https://namu.wiki/w/%EC%A7%80%EC%96%B8%ED%95%99',
 'https://namu.wiki/w/%EB%B8%8C%EB%A0%8C%ED%8A%B8%ED%8F%AC%EB%93%9C%20FC/2021-22%20%EC%8B%9C%EC%A6%8C/%EB%A6%AC%EA%B7%B8',
 'https://namu.wiki/w/%EA%B4%B4%EB%AC%BC%EC%82%AC%EB%B3%80',
 'https://namu.wiki/w/%ED%86%A0%ED%8A%B8%EB%84%98%20%ED%99%8B%EC%8A%A4%ED%8D%BC%20FC/2021-22%20%EC%8B%9C%EC%A6%8C/%EB%A6%AC%EA%B7%B8',
 'https://namu.wiki/w/%EA%B9%80%EC%A7%80%ED%98%84(%EC%B6%95%EA%B5%AC%EC%84%A0%EC%88%98)',
 'https://namu.wiki/w/%EC%8A%A4%EB%B2%A4%20%EB%B3%B4%ED%8A%B8%EB%A7%8C',
 'https://namu.wiki/w/%EC%A0%84%EC%A7%80%EC%A0%81%20%EB%8F%85%EC%9E%90%20%EC%8B%9C%EC%A0%90/%EB%93%B1%EC%9E%A5%EC%9D%B8%EB%AC%BC',
 'https://namu.wiki/w/%EB%B8%8C%EB%9D%BC%EC%9D%B4%EC%96%B8%20%ED%83%80%EC%9D%BC%EB%9F%AC',
 'https://namu.wiki/w/%EA%B3%A8%EB%93%A0%EB%94%94%EC%8A%A4%ED%81%AC%EC%96%B4%EC%9B%8C%EC%A6%88',
 'https://namu.wiki/w/%EC%A0%84%EB%AC%B8%EA%B0%80%EB%AC%BC',
 'https://namu.wiki/w/FUN!%20FUN!%20FANTAST

### [각 링크 페이지내 텍스트 구조를 확인하여 제목, 카테고리, 내용 출력]

In [6]:
# 윈도우용 크롬 웹드라이버 실행 경로 (Windows) 지정
excutable_path = "chromedriver.exe"
driver = webdriver.Chrome(executable_path=excutable_path)
# 크롬 드라이버를 통해 page_urls[0]번째 사이트의 HTML 문서 가져옴
driver.get(page_urls[0])  # page_urls[0] 의 정보를 가져옴
req = driver.page_source # 페이지 소스를 req에 저장
soup = BeautifulSoup(req, 'html.parser') # html.parser로 파싱
contents_table = soup.find(name="article") #  불러온 소스에서 태그명이 article인 요소 하나만 추출

### 타이틀 추출
title = contents_table.find_all('h1')[0] # 태그명이 h1인 모든 태그 추출, article h1

### 카테고리 추출
category = contents_table.find_all('ul')[0]

### 내용 추출
#contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})  
#div  태그 중  class 속성값이  wiki-paragraph인 요소를 추출
content_paragraphs = contents_table.select("div.wiki-paragraph")  

#  내용으로 추출한 리스트를 하나의 문자열로 전처리
content_corpus_list = [] # 내용 중 텍스트만 담을 빈 리스트 생성
# content_paragraphs 리스트의 값을 순서대로 paragraphs에 대입
for paragraphs in content_paragraphs: # content_paragraphs 리스트의 값을 순서대로 paragraphs에 대입
    content_corpus_list.append(paragraphs.text)  # 가져온 결과 태그 중 텍스트만 추출하여 content_corpus_list에 추가
content_corpus =" ".join(content_corpus_list) #"텍스트".join(리스트명) => 리스트의 요소를 "텍스트"로 구분하여 하나의 문자열로 만듦

print(title.text) # 제목 출력
print("\n")
print(category.text) # 카테고리 출력
print("\n")
print(content_corpus) # 내용 출력

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

휘장 


휘장토막글/사회


이 문서는 450자 이하의 토막글입니다.틀 적용 시 분류:토막글의 하위 분류 중 알맞은 곳에 분류해 주세요. 대한민국의 포상제도 훈장포장기장표창휘장 ※ 약장  대한민국의 포상제도  훈장 포장 기장 표창 휘장 ※ 약장  대한간호협회 지정 간호사 휘장[1] 1. 개요2. 휘장 분배3. 포상 제도로서의 휘장4. 같이보기 휘장(徽章)은 신분이나 직무, 명예 따위를 나타내는 띠, 리본, 배지 따위의 표지(標識)[출처] 또는 국가, 단체를 상징하는 표시를 뜻한다. '휘장 분배'에서 '휘장'은 신분이나 지위, 명예를 나타내기 위해 모자나 의복에 붙이는 표(뱃지, badge)를 가리킨다. 그리고 분배는 '고르게 나누어 줌'이란 뜻이다. 따라서 '휘장 분배'는 총회나 노회에서 새로 당선된 임원들에게 '꽃'을 달아주는 축하 의식을 말한다. 대한민국의 포상 제도에서 훈장, 포장, 기장, 표창에 이은 다섯 번째가 바로 휘장이다. 일반인들이 받기 상대적으로 쉬운 표창과 달리 휘장은 특정 직업이 아닌 한 받기 어렵다. 그럼에도 우리가 생각지도 못한 곳에서 포상 성격을 가진 휘장을 목격할 수 있다. 다만, 우리가 잘 느끼지 못할 뿐이다.대표적으로 군인, 경찰관, 소방관, 자치경찰, 철도경찰, 관세직/교정직/출입국관리직 공무원[3]의 옷에 달려 있는 약장을 떠올릴 수 있다. 약장도 휘장의 하나로서 기능하며, 훈장, 포장, 기장, 표창을 수여 받을 경우 대부분 부속품으로서 약장이 따라 온다. 물론 약장 하나만 받을 수도 있으며, 이런 경우는 대개 표창을 수여 받기 애매한 공적이나 명예에 대한 포상으로서 기능한다.사업용 자동차 운전자들이 받는 무사고 표시장도 하나의 휘장이다. 이 휘장은 10년~30년 동안 무사고로 사업용 자동차를 운전한 사람들에게 주어지며, 이것을 받은 사람들은 자신의 자동차에 장식용으로 진열해 두거나, 모범운전자일 경우 자신의 상의에 부착하기도 한다. 이 무사고 표시장도 도로교통법에 명시된 하나의 포상이라고 할 수 있다. 전투보병휘장 지휘관 

### [각각 링크 페이지를 크롤링하여 제목, 카테고리, 내용 출력]

In [7]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비
columns = ["title", "category", "content_text"]
df = pd.DataFrame(columns=columns)

#for page_url in page_urls:
for i in range(10):
    # 윈도우용 크롬 웹드라이버 실행 경로 (Windows) 지정
    excutable_path = "chromedriver.exe"
    driver = webdriver.Chrome(executable_path=excutable_path)
    # 크롬 드라이버를 통해 page_urls[0]번째 사이트의 HTML 문서 가져옴
    #driver.get(page_url)  # page_urls[i],  page_url의 정보를 가져옴
    driver.get(page_urls[i])  # page_urls[i],  page_url의 정보를 가져옴
    req = driver.page_source # 페이지 소스를 req에 저장
    soup = BeautifulSoup(req, 'html.parser') # html.parser로 파싱
    contents_table = soup.find(name="article") #  불러온 소스에서 태그명이 article인 요소 하나만 추출

    ### 타이틀 추출
    title = contents_table.find_all('h1')[0] # 태그명이 h1인 모든 태그 추출, article h1
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
        
    ### 카테고리 추출
    # 카테고리 정보가 없는 경우를 확인합니다.
    if len(contents_table.find_all("ul")) > 0: # article ul 로 검색한 결과 여러 ul 결과가 나올 경우
        category = contents_table.find_all("ul")[0] # 제일 첫번째 article ul 을 category로 설정
    else:
        category = None
        
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""

    ### 내용 추출
    #contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})  
    #div  태그 중  class 속성값이  wiki-paragraph인 요소를 추출
    content_paragraphs = contents_table.select("div.wiki-paragraph")  
    #  내용으로 추출한 리스트를 하나의 문자열로 전처리
    content_corpus_list = [] # 내용 중 텍스트만 담을 빈 리스트 생성
    
    # content_paragraphs 리스트의 값을 순서대로 paragraphs에 대입
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")

    # 모든 정보를 하나의 데이터 프레임에 저장하기 위해서 시리즈 생성
    # 각 페이지의 정보를 추출하여 제목, 카테고리, 내용 순으로 행을 생성
    row = [row_title, row_category, "".join(content_corpus_list)]
    # 시리즈로 만듦
    series = pd.Series(row, index=df.columns)
    # 데이터 프레임에 시리즈를 추가, 한 페이지 당 하나의 행 추가
    df = df.append(series, ignore_index=True)
    
    # 크롤링에 사용한 브라우저를 종료합니다.
    driver.close()

In [8]:
# 데이터 프레임을 출력합니다.
df

,title,category,content_text
0,휘장,휘장토막글/사회,이 문서는 450자 이하의 토막글입니다.틀 적용 시 분류:토막글의 하위 분류 중 알...
1,지언학,"1994년 출생대한민국의 축구선수구미시 출신 인물공격수AD 알코르콘/은퇴, 이적한수...",김천 상무 FC 2022 시즌 스쿼드 [ 펼치기 · 접기 ] 단장 이흥실 · 감독 ...
2,브렌트포드 FC/2021-22 시즌/리그,브렌트포드 FC,상위 문서: 브렌트포드 FC/2021-22 시즌 브렌트포드 FC 시즌별 정규...
3,괴물사변,2017년 만화일본 만화/목록,"괴물사변怪物事変장르요괴, 다크 판타지, 배틀, 서스펜스작가아이모토 쇼출판사 슈에이샤..."
4,토트넘 홋스퍼 FC/2021-22 시즌/리그,토트넘 홋스퍼 FC/역대 시즌,상위 문서: 토트넘 홋스퍼 FC/2021-22 시즌 토트넘 홋스퍼 FC 정규...
5,김지현(축구선수),"대한민국의 축구선수공격수1996년 출생제주특별자치도 출신 인물강원 FC/은퇴, 이적...",은(는) 여기로 연결됩니다. 동명이인에 대한 내용은 김지현 문서를의 번 문단을...
6,스벤 보트만,"네덜란드의 축구선수AFC 아약스/은퇴, 이적LOSC 릴/현역",LOSC 릴 2021-22 시즌 선수 명단1 그르비치 · 2 첼리크 · 3 티...
7,전지적 독자 시점/등장인물,전지적 독자 시점/등장인물,가입 후 15일이 지나야 편집 가능한 문서입니다. 상위 문서: 전지적 독자...
8,브라이언 타일러,미국 작곡가미국의 영화 음악가1972년 출생로스앤젤레스 출신 인물UCLA 출신하버드...,브라이언 타일러Brian Tyler본명Brian Theodore Tyler브라이언 ...
9,골든디스크어워즈,골든디스크어워즈,대한민국의 대중음악 시상식의 시상식 [ 펼치기 · 접기 ]골든디스크어워즈백상예술대상...


# [명사만을 추출하여 워드 클라우드 그리기]

## [코엔엘파이(konlpy)를 이용한 형태소 분석]

- 품사란 단어를 기능, 형태, 의미에 따라 나눈 갈래
- 우리나라의 학교 문법에서는 명사, 대명사, 수사, 조사, 동사, 형용사, 관형사, 부사, 감탄사의 아홉 가지로 분류

### [형태소 분석과 품사 태깅]
- 형태소 : 더 이상 분리를 할 수 없는 의미를 갖는최소 단어를 의미
- 형태소 분석 : 형태소를 비롯하여, 어근, 접두사/접미사, 품사(POS, part-of-speech) 등 다양한 언어적 속성의 구조를 파악하는 것
- 품사 태깅 : 형태소와 품사를 매칭시키는 것

### [빈도 분석: 문장 형태소 분석 - KoNLPy]
- KoNLPy : 파이썬 한국어 형태소 분석 라이브러리

# <Step2. 추출> : 키워드 추출
### [텍스트 데이터 전처리] 정규식을 사용하여 한글과 띄어쓰기만 가져오기
#### 파이썬 정규표현식(re) 사용법 - 05. 주석, 치환, 분리
- 정규표현식: 컴파일 => re.compile , 컴파일을 미리 해 두고 이를 저장
- 정규표현식: 치환 => re.sub(pattern, repl, string, count, flags)

In [9]:
# 한글 코드 범위
# ㄱ ~ ㅎ: 0x3131 ~ 0x314e
# ㅏ ~ ㅣ: 0x314f ~ 0x3163
# 가 ~ 힣: 0xac00 ~ 0xd7a3
#  [^ㄱ-|가-힣+] 한글과 띄어쓰기의 정규식 패턴
# 사용자 정의 함수 선언
def text_cleaning(text):
    # 한글과 띄어쓰기를 제외한 모든 글자 패턴을 지정하여 hangul로 정의
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 
    result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 글자 패턴을 '' 빈 문자로 치환
    return result

In [10]:
df['title'][0]

'휘장 '

In [11]:
print(text_cleaning(df['title'][0]))

휘장 


In [12]:
# "title", "category", "content_text"]
df['category'][0]

'휘장토막글/사회'

In [13]:
print(text_cleaning(df['category'][0]))

휘장토막글사회


In [14]:
df['content_text'][0]

"이 문서는 450자 이하의 토막글입니다.틀 적용 시 분류:토막글의 하위 분류 중 알맞은 곳에 분류해 주세요. 대한민국의 포상제도 훈장포장기장표창휘장 ※ 약장 대한민국의 포상제도 훈장포장기장표창휘장 ※ 약장대한간호협회 지정 간호사 휘장[1]1. 개요2. 휘장 분배3. 포상 제도로서의 휘장4. 같이보기휘장(徽章)은 신분이나 직무, 명예 따위를 나타내는 띠, 리본, 배지 따위의 표지(標識)[출처] 또는 국가, 단체를 상징하는 표시를 뜻한다.'휘장 분배'에서 '휘장'은 신분이나 지위, 명예를 나타내기 위해 모자나 의복에 붙이는 표(뱃지, badge)를 가리킨다. 그리고 분배는 '고르게 나누어 줌'이란 뜻이다. 따라서 '휘장 분배'는 총회나 노회에서 새로 당선된 임원들에게 '꽃'을 달아주는 축하 의식을 말한다.대한민국의 포상 제도에서 훈장, 포장, 기장, 표창에 이은 다섯 번째가 바로 휘장이다. 일반인들이 받기 상대적으로 쉬운 표창과 달리 휘장은 특정 직업이 아닌 한 받기 어렵다. 그럼에도 우리가 생각지도 못한 곳에서 포상 성격을 가진 휘장을 목격할 수 있다. 다만, 우리가 잘 느끼지 못할 뿐이다.대표적으로 군인, 경찰관, 소방관, 자치경찰, 철도경찰, 관세직/교정직/출입국관리직 공무원[3]의 옷에 달려 있는 약장을 떠올릴 수 있다. 약장도 휘장의 하나로서 기능하며, 훈장, 포장, 기장, 표창을 수여 받을 경우 대부분 부속품으로서 약장이 따라 온다. 물론 약장 하나만 받을 수도 있으며, 이런 경우는 대개 표창을 수여 받기 애매한 공적이나 명예에 대한 포상으로서 기능한다.사업용 자동차 운전자들이 받는 무사고 표시장도 하나의 휘장이다. 이 휘장은 10년~30년 동안 무사고로 사업용 자동차를 운전한 사람들에게 주어지며, 이것을 받은 사람들은 자신의 자동차에 장식용으로 진열해 두거나, 모범운전자일 경우 자신의 상의에 부착하기도 한다. 이 무사고 표시장도 도로교통법에 명시된 하나의 포상이라고 할 수 있다.전투보병휘장지휘관 휘장"

In [15]:
print(text_cleaning(df['content_text'][0]))

이 문서는 자 이하의 토막글입니다틀 적용 시 분류토막글의 하위 분류 중 알맞은 곳에 분류해 주세요 대한민국의 포상제도 훈장포장기장표창휘장  약장 대한민국의 포상제도 훈장포장기장표창휘장  약장대한간호협회 지정 간호사 휘장 개요 휘장 분배 포상 제도로서의 휘장 같이보기휘장은 신분이나 직무 명예 따위를 나타내는 띠 리본 배지 따위의 표지출처 또는 국가 단체를 상징하는 표시를 뜻한다휘장 분배에서 휘장은 신분이나 지위 명예를 나타내기 위해 모자나 의복에 붙이는 표뱃지 를 가리킨다 그리고 분배는 고르게 나누어 줌이란 뜻이다 따라서 휘장 분배는 총회나 노회에서 새로 당선된 임원들에게 꽃을 달아주는 축하 의식을 말한다대한민국의 포상 제도에서 훈장 포장 기장 표창에 이은 다섯 번째가 바로 휘장이다 일반인들이 받기 상대적으로 쉬운 표창과 달리 휘장은 특정 직업이 아닌 한 받기 어렵다 그럼에도 우리가 생각지도 못한 곳에서 포상 성격을 가진 휘장을 목격할 수 있다 다만 우리가 잘 느끼지 못할 뿐이다대표적으로 군인 경찰관 소방관 자치경찰 철도경찰 관세직교정직출입국관리직 공무원의 옷에 달려 있는 약장을 떠올릴 수 있다 약장도 휘장의 하나로서 기능하며 훈장 포장 기장 표창을 수여 받을 경우 대부분 부속품으로서 약장이 따라 온다 물론 약장 하나만 받을 수도 있으며 이런 경우는 대개 표창을 수여 받기 애매한 공적이나 명예에 대한 포상으로서 기능한다사업용 자동차 운전자들이 받는 무사고 표시장도 하나의 휘장이다 이 휘장은 년년 동안 무사고로 사업용 자동차를 운전한 사람들에게 주어지며 이것을 받은 사람들은 자신의 자동차에 장식용으로 진열해 두거나 모범운전자일 경우 자신의 상의에 부착하기도 한다 이 무사고 표시장도 도로교통법에 명시된 하나의 포상이라고 할 수 있다전투보병휘장지휘관 휘장


In [16]:
# 각 피처마다 데이터 전처리를 적용
# 각 피처마다 한글과 띄어씌기를 제외한 모든 부분을 제가
df['title'] = df['title'].apply(lambda x:text_cleaning(x))
df['category'] = df['category'].apply(lambda x:text_cleaning(x))
df['content_text'] = df['content_text'].apply(lambda x:text_cleaning(x))

df.head(5)

,title,category,content_text
0,휘장,휘장토막글사회,이 문서는 자 이하의 토막글입니다틀 적용 시 분류토막글의 하위 분류 중 알맞은 곳에...
1,지언학,년 출생대한민국의 축구선수구미시 출신 인물공격수 알코르콘은퇴 이적한수원 축구단은퇴 ...,김천 상무 시즌 스쿼드 펼치기 접기 단장 이흥실 감독 김태완 수석 코치...
2,브렌트포드 시즌리그,브렌트포드,상위 문서 브렌트포드 시즌 브렌트포드 시즌별 정규 리그 시즌 시즌 시즌 프...
3,괴물사변,년 만화일본 만화목록,괴물사변장르요괴 다크 판타지 배틀 서스펜스작가아이모토 쇼출판사 슈에이샤 학산문화사연...
4,토트넘 홋스퍼 시즌리그,토트넘 홋스퍼 역대 시즌,상위 문서 토트넘 홋스퍼 시즌 토트넘 홋스퍼 정규 시즌리그 시즌 시즌 시...


[말뭉치 만들기]

In [17]:
# 각 피처마다 말뭉치를 생성
df['title'].tolist()

['휘장 ',
 '지언학 ',
 '브렌트포드  시즌리그 ',
 '괴물사변 ',
 '토트넘 홋스퍼  시즌리그 ',
 '김지현축구선수 ',
 '스벤 보트만 ',
 '전지적 독자 시점등장인물 ',
 '브라이언 타일러 ',
 '골든디스크어워즈 ']

In [18]:
title_corpus = ''.join(df['title'].tolist())
category_corpus = ''.join(df['category'].tolist())
content_corpus = ''.join(df['content_text'].tolist())

print(title_corpus)
print(category_corpus)

휘장 지언학 브렌트포드  시즌리그 괴물사변 토트넘 홋스퍼  시즌리그 김지현축구선수 스벤 보트만 전지적 독자 시점등장인물 브라이언 타일러 골든디스크어워즈 
휘장토막글사회년 출생대한민국의 축구선수구미시 출신 인물공격수 알코르콘은퇴 이적한수원 축구단은퇴 이적김해시청 축구단은퇴 이적인천 유나이티드 은퇴 이적경희대학교 출신상무 축구단현역브렌트포드 년 만화일본 만화목록토트넘 홋스퍼 역대 시즌대한민국의 축구선수공격수년 출생제주특별자치도 출신 인물강원 은퇴 이적울산 현대은퇴 이적상무 축구단현역제주제일고등학교 출신한라대학교 출신네덜란드의 축구선수 아약스은퇴 이적 릴현역전지적 독자 시점등장인물미국 작곡가미국의 영화 음악가년 출생로스앤젤레스 출신 인물 출신하버드 대학교 출신골든디스크어워즈


In [20]:
import konlpy

In [25]:
from konlpy.tag import Okt
from collections import Counter

# konlpy의 형태 분석기로 명사 단위의 키워드를 추출합니다.
nouns_tagger =Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)

In [26]:
content_corpus

'이 문서는 자 이하의 토막글입니다틀 적용 시 분류토막글의 하위 분류 중 알맞은 곳에 분류해 주세요 대한민국의 포상제도 훈장포장기장표창휘장  약장 대한민국의 포상제도 훈장포장기장표창휘장  약장대한간호협회 지정 간호사 휘장 개요 휘장 분배 포상 제도로서의 휘장 같이보기휘장은 신분이나 직무 명예 따위를 나타내는 띠 리본 배지 따위의 표지출처 또는 국가 단체를 상징하는 표시를 뜻한다휘장 분배에서 휘장은 신분이나 지위 명예를 나타내기 위해 모자나 의복에 붙이는 표뱃지 를 가리킨다 그리고 분배는 고르게 나누어 줌이란 뜻이다 따라서 휘장 분배는 총회나 노회에서 새로 당선된 임원들에게 꽃을 달아주는 축하 의식을 말한다대한민국의 포상 제도에서 훈장 포장 기장 표창에 이은 다섯 번째가 바로 휘장이다 일반인들이 받기 상대적으로 쉬운 표창과 달리 휘장은 특정 직업이 아닌 한 받기 어렵다 그럼에도 우리가 생각지도 못한 곳에서 포상 성격을 가진 휘장을 목격할 수 있다 다만 우리가 잘 느끼지 못할 뿐이다대표적으로 군인 경찰관 소방관 자치경찰 철도경찰 관세직교정직출입국관리직 공무원의 옷에 달려 있는 약장을 떠올릴 수 있다 약장도 휘장의 하나로서 기능하며 훈장 포장 기장 표창을 수여 받을 경우 대부분 부속품으로서 약장이 따라 온다 물론 약장 하나만 받을 수도 있으며 이런 경우는 대개 표창을 수여 받기 애매한 공적이나 명예에 대한 포상으로서 기능한다사업용 자동차 운전자들이 받는 무사고 표시장도 하나의 휘장이다 이 휘장은 년년 동안 무사고로 사업용 자동차를 운전한 사람들에게 주어지며 이것을 받은 사람들은 자신의 자동차에 장식용으로 진열해 두거나 모범운전자일 경우 자신의 상의에 부착하기도 한다 이 무사고 표시장도 도로교통법에 명시된 하나의 포상이라고 할 수 있다전투보병휘장지휘관 휘장김천 상무   시즌 스쿼드  펼치기  접기  단장 이흥실  감독 김태완  수석 코치 성한수  코치 김치우 신상우   코치 곽상득  의무 트레이너 구성훈  전력분석관 이건목 황인재   하창래   송주훈   최준혁   

In [28]:
count

Counter({'이': 349,
         '문서': 93,
         '자': 32,
         '이하': 5,
         '토막글': 2,
         '틀': 7,
         '적용': 2,
         '시': 38,
         '분류': 5,
         '하위': 10,
         '중': 196,
         '곳': 28,
         '대한민국': 14,
         '포상': 7,
         '제도': 7,
         '훈장': 4,
         '포장': 4,
         '기장': 4,
         '표창': 6,
         '휘장': 17,
         '약장': 6,
         '간호': 1,
         '협회': 1,
         '지정': 2,
         '간호사': 1,
         '개요': 10,
         '분배': 5,
         '로서': 11,
         '보기': 10,
         '신분': 4,
         '직무': 1,
         '명예': 4,
         '따위': 7,
         '띠': 1,
         '리본': 1,
         '배지': 1,
         '표지': 3,
         '출처': 4,
         '국가': 3,
         '단체': 4,
         '상징': 2,
         '표시': 4,
         '뜻': 12,
         '지위': 1,
         '위해': 95,
         '모자': 5,
         '의복': 1,
         '표': 14,
         '뱃지': 1,
         '를': 52,
         '줌': 1,
         '따라서': 8,
         '총회': 1,
         '노회': 1,
         '당선': 1

[키워드 가다듬기]

In [30]:
remove_char_counter=Counter({x:count[x] for x in count if len(x)>1})
print(remove_char_counter)

Counter({'경기': 773, '독자': 570, '토트넘': 426, '슈팅': 359, '성좌': 279, '이후': 276, '공격': 268, '리그': 238, '선수': 237, '패스': 224, '라운드': 217, '브렌트포드': 216, '수비': 206, '모습': 206, '시즌': 185, '시나리오': 184, '상황': 175, '자신': 167, '손흥민': 165, '때문': 164, '퇴장': 154, '감독': 149, '상대': 143, '경고': 142, '정도': 142, '코너킥': 140, '점유': 139, '유효': 133, '케인': 132, '진명': 132, '파울': 131, '스코어': 129, '오프사이드': 127, '카바네': 127, '공률': 126, '등장': 124, '기록': 121, '부상': 117, '중혁': 113, '설화': 110, '하나': 107, '마왕': 105, '언급': 103, '누누': 103, '모두': 98, '승리': 98, '홋스퍼': 97, '시작': 96, '후반': 96, '매치': 96, '위해': 95, '문서': 93, '또한': 93, '가장': 91, '시간': 90, '교체': 89, '이번': 89, '압박': 86, '사실': 85, '도깨비': 85, '원정': 82, '주심': 82, '크리스': 81, '득점': 81, '전술': 81, '다른': 81, '다시': 80, '결과': 80, '크로스': 79, '리버풀': 79, '관중': 78, '하이라이트': 78, '음반': 77, '마지막': 76, '첼시': 76, '리포트': 76, '대상': 74, '선발': 74, '미정': 73, '이전': 72, '문단': 72, '접기': 70, '기회': 70, '본인': 70, '예상': 69, '일행': 69, '배후': 69, '신화': 69, '출전': 68, '아스날': 68, '스타디움': 68, '결석': 68, 

[불영어 제거]
- 실질적인 의미가 없는 키워드 처리
- 관사나 접속사 등 실질적인 의마가 없으면서 동시에 의미적인 독립을 할수 없는 품사 제거
- 한국어 약식 불용어 사전 예시 파입입니다. 출처 (http://www.ranks.nl/stopwords/korean)
- 인터넷 검색 시 검색 용어로 사용하지는 단어, 관사, 전치사, 조사, 접속사 등은 검색 색인 단어로 의마가 없는 단어
- 그러나, 각 검색 엔진마다 내용은 다를 수도 있음

In [31]:
korean_stopwords_path = 'korean_stopwords.txt'

# 텍스트 파일을 오픈 
with open(korean_stopwords_path,encoding = 'utf8') as f: # 불용어 텍스트 파일을 열어 f로 치환
    stopwords = f.readlines() # 파일로 불용어를 한줄씩 읽어들임
stopwords = [x.strip() for x in stopwords] # 리스트 생성
print(stopwords[:10]) #리스트 출력

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']
